# 注意力评分函数
观察函数$\sum_{i=1}^{n}\text{softmax}(-\frac{1}{2}(x-x_i)^2)y_i = \sum_{i}\alpha(x, x_i)y_i$

其中
- 注意力分数：$-\frac{1}{2}(x-x_i)^2$。即softmax函数的自变量
- 注意力权重：$\alpha(x, x_i)$。即softmax函数的输出结果

**文章关心注意力分数是如何设置的**

query $\bf{q} \in \mathbb{R}^q$，$m$对key-value $(\bf{k}_1,\bf{v}_1)$，其中$\bf{k}_i \in \mathbb{R}^k$，$\bf{v}_i \in \mathbb{R}^v$。（即维度可以完全不同）
注意力池化层：


In [7]:
import math
import torch
from torch import nn
from d2l import torch as d2l

## 1. 隐蔽softmax操作
为了仅将有意义的词元作为值来获取注意力汇聚， 可以指定一个有效序列长度（即词元的个数）， 以便在计算softmax时过滤掉超出指定范围的位置。其中任何超出有效长度的位置都被掩蔽并置为0。

In [8]:
#@save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)

In [9]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

tensor([[[0.3903, 0.6097, 0.0000, 0.0000],
         [0.3268, 0.6732, 0.0000, 0.0000]],

        [[0.3807, 0.3787, 0.2406, 0.0000],
         [0.3635, 0.3510, 0.2855, 0.0000]]])

In [10]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.2416, 0.4330, 0.3253, 0.0000]],

        [[0.3259, 0.6741, 0.0000, 0.0000],
         [0.2175, 0.3473, 0.2248, 0.2104]]])